In [ ]:
#= Hello.

Let me introduce you to some new features of the Skyrmions3D package. I've also updated the names of some things. First, the pion field field of your Skyrmion is now called pion_field. So you can explore the field by running Skyrmion.pion_field. The rational map function is now called makeRationalMap, insteadof makeRM.

Let's import the Skyrmions3D and GLMakie packages and try and convince VScode/Jupyter notebook to *use* GLMakie (ie put plots in a pop-out box)

=# 

In [ ]:
]add https://github.com/chrishalcrow/Skyrmions3D.jl.git

In [ ]:
using Skyrmions
using GLMakie
GLMakie.activate!()
Makie.inline!(false)

In [ ]:
# Now let's make a skyrmion, and have a look at it

my_skyrmion = Skyrmion(40, 0.2)

p4(z) = z^4 + 3.0*sqrt(3.0)*im*z^2 + 1.0
q4(z) = z^4 - 3.0*sqrt(3.0)*im*z^2 + 1.0
f4(r) = pi*exp( -(r.^3)./12.0 )

makeRationalMap!(my_skyrmion, f4, p4, q4)

# If this doesn't work, try display(GLMakie.Screen(),plot_baryon_density(my_skyrmion,iso_value=3.0) instead
display(plot_baryon_density(my_skyrmion,iso_value=2.0))

In [ ]:
# We can alter some of the skyrmion's properties by as follows

my_skyrmion.mpi = 1.0
my_skyrmion.periodic = true
Energy(my_skyrmion)

# That's right, we can now study periodic skyrmions. You can always check all the available properties 
# of a `Skyrmion` struct by running `?Skyrmion'

In [ ]:
# There is also now some dynamics. There are *two* available functions. First, you can gradient flow:

gradient_flow!(my_skyrmion, steps=100)
Energy(my_skyrmion)

In [ ]:
# Nice, our energy has gone down. With the gradient_flow! function, you can either flow for a fixed number 
# of steps, as above. Or you can run until your error function goes below some fixed tolerance. This can be 
# done by adding `tolerance = ??` instead of steps. For instance, the code below will only stop once the sum
# of (dEdπ)^2 over all lattice sites is less than 1.0. I think the user should be allowed to pick their own 
# tolerance function. Any ideas of which ones are good, anyone?

gradient_flow!(my_skyrmion, tolerance = 100.0)
display(plot_baryon_density(my_skyrmion,iso_value=2.0))
Energy(my_skyrmion)

In [ ]:
# Another option is to use Arrested Newton Flow. Here we solve ddot(π) = -dEdπ, but arrest the momentum if 
# the energy of the field increases. I also find it to converge faster if we apply a short gradient_flow 
# whenever we arrest. So, this isn't `pure` Arrested Newton Flow. We also need to supply an initial skyrmion
# velocity:

skd = zeros(my_skyrmion.lp[1], my_skyrmion.lp[2], my_skyrmion.lp[3], 4);
arrested_newton_flow!(my_skyrmion, skd, tolerance = 1.0, print_stuff=true)
display(plot_baryon_density(my_skyrmion,iso_value=4.0))
Energy(my_skyrmion)

In [ ]:
# Cool. Another nice new feature is that we can resize the lattice. Suppose we're interested in the periodic 
# B=4 solution. We might want to squeeze the box a bit. We can do this using the resize_lattice! function

resize_lattice!(my_skyrmion,[30,30,30],[0.2,0.2,0.2])

# Let's flow again...

skd = zeros(my_skyrmion.lp[1], my_skyrmion.lp[2], my_skyrmion.lp[3], 4);
arrested_newton_flow!(my_skyrmion, skd, steps=500, print_stuff=true)

In [ ]:
# But 0.2 lattice spacing is a bit big. Let's reduce it, but compensate with more points

resize_lattice!(my_skyrmion,[60,60,60],[0.1,0.1,0.1])
skd = zeros(my_skyrmion.lp[1], my_skyrmion.lp[2], my_skyrmion.lp[3], 4);
arrested_newton_flow!(my_skyrmion, skd, tolerance = 10.0)
plot_baryon_density(my_skyrmion,iso_value=3.0)

In [ ]:
# So, that's all good. But let's see something more FUN! I'll make a new skyrmion and start the function 

my_skyrmion = Skyrmion(40, 0.2, mpi=1.0, periodic=false);
makeRationalMap!(my_skyrmion, f4, p4, q4)

interactive_flow(my_skyrmion)